# Investigating the need for weighting / normalization vis-a-vis variance averaging

## synopsis

During the MoDaCor hackathon, the need for an additional normalisation matrix was argued by Jerome Kieffer. This document sets out to investigate and clarify that need, from both a mathematical as well as an empirical point of view. 

## Introduction

MoDaCor's core concept is that we can chain together modular operations in arbitrary sequence. The data ('BaseData' instance) is modified in place by each module, and generally does not change shape. Averaging operations are the exception to this rule, where the dimensionality of data can be changed during this operation. 

MoDaCor also propagates (a list of) uncertainty estimators in the form of variances (variances, i.e. the square of the uncertainties, are much faster to propagate). These variances can originate from multiple estimation sources. For example, the counting (Poisson) variance can be estimated at the start, but during an averaging operation an additional variance estimate can be gained from the square of the standard error on the mean (SEM). Further variance estimators may be added along the operations. Each should be propagated independently through the modules, until such a time that a combined estimator is desired from these individual estimators. 

All data must be accompanied with at least one variance estimator, as well as units ('dimensionless' if it does not have a unit). Operations with scalars (with uncertainties and units) are applied to the BaseData scalar rather than the signal matrix. 

The question now is whether BaseData's 'signal' matrix should be accompanied by an identically-sized 'normalization' or 'weighting' matrix, and how to deal with it in operations. The argument stems from the propagation of uncertainties in averaging operations, where sample size should likely be taken into consideration. This sample size is different per datapoint, and can be set, depending on factors such as detector pixel dimensions, number of pixels in an averaging bin, etc. 

For this document, we will not consider 'pixel splitting' operations, and consider each datapoint to be an indivisible unit. 

## investigating the effect of a weighting matrix. 

### The argument from different pixel dimensions

Imagine we have a detector with two pixels, one of which has ten times the area of the other. pixel 1 has area $A_1 = 1$ with a variance $\sigma^2_{A,1} = 0.01$, pixel 2 an area of $A_2 = 10$ with a variance $\sigma^2_{A,2} = 0.04$. We are counting photons, so counting statistics apply. 

The first pixel collects $I_1 = 9$ counts, the second collects $I_2 = 100$ counts in the same period of time. The counting statistics-based uncertainties are therefore $\sigma_1 = \sqrt(I_1) = 3$ and $\sigma_2 = \sqrt(I_2) = 10$ counts, their variances $\sigma^2_1 = u_1^2 = 9$ and $\sigma^2_2 = u_2^2 = 100$, respectively. 

We do two steps to this data: one by area, and an averaging step, averaging over the two pixels. 

The normalization by pixel area $A$ has the following effect on the signal $I$ and the variance $\sigma^2$:
$$ 
I_{i, \mathrm{out}} = I_{i, \mathrm{in}} / A_i \quad\forall \in \set{1, ..., n}
$$
and using Goodman's expression [4] for the multiplication operations for the uncorrelated case: 
$$
\sigma^2_{i, \mathrm{out}} = \sigma^2_{i, \mathrm{in}} A_i^2 + \sigma^2_{A,i} I_{i, \mathrm{in}}^2 + \sigma^2_{i, \mathrm{in}} \sigma^2_{A,i} \quad\forall \in \set{1, ..., n}
$$
which, when also tracking optional weighting (by relative area in this case), would lead to: 


In [96]:
# numerical example, setup
import numpy as np

w  = np.array([1, 1], dtype=float) # weights
w_unweighted  = np.array([1, 1], dtype=float) # weights matrix to be used for unweighted averages
I  = np.array([9, 100], dtype=float) # signal
v  = np.array([9, 100], dtype=float) # variance of signal
A  = np.array([1, 10], dtype=float) # area to normalize by
vA = np.array([0.01, 0.04], dtype=float) # variance of area

In [ ]:

# step 1
I_out = I / A
f = 1/A # multiplier for Goodman's algo
vf = f**2 * (np.sqrt(vA)/A)**2 # variance of the multiplier for Goodman's algo
w *= A
v_goodman = v * f**2 + vf * I**2 + v*vf
v = I_out**2 * ((np.sqrt(vA) / A)**2 + (np.sqrt(v)/ I)**2) # ignoring cross-term
I=I_out

In [98]:
[print(f'pixel: {i}: signal I: {I[i]}, variance v: {v[i]}, goodman variance v: {v_goodman[i]}, weight w: {w[i]}') for i in range(len(I))]

pixel: 0: signal I: 9.0, variance v: 9.81, goodman variance v: 9.9, weight w: 1.0
pixel: 1: signal I: 10.0, variance v: 1.04, goodman variance v: 1.0404000000000002, weight w: 10.0


[None, None]


|       | signal $I$ | variance $v$ | weighting $w_i$ |
| ----- | ---------- | ------------ | --------------- |
|pixel 1| 9          | 9.9          | 1               |
|pixel 2| 10         | 1.04         | 10              |

The average signal $\hat{I}$ would then follow from equation 12 in [3]: 

$$ 
\hat{I} = \frac{1}{\sum_i w_i} \sum_i w_i I_i
$$
a new variance estimator $\sigma^2_{\mathrm{av}}$ is obtained from equation 13 in [3] (note: not sure why the original paper calls this a "(Co)variance", probably because of the mixin of weights?): 

(note, this seems weirdly lacking in squared weights.)
$$
\sigma^2_{\mathrm{av}} = \frac{1}{\sum_i w_i} \sum_i w_i (I_i - \hat{I}) (w_i - \hat{w})
$$

and the propagated variance should be:

$$ 
\sigma^2 = \frac{1}{\sum_i w_i^2} \sum_i w_i^2 \sigma^2_{i, \mathrm{in}}
$$


for the simple average, $w_i = 1$, for the weighted average, we would assume the weighting in the previous table. 

The numerical example then becomes: 


In [99]:
w_av = w

I_hat = 1/w_av.sum() * (w_av * I).sum()
v_hat = 1/(w_av**2).sum() * (w_av**2 * v).sum()
v_est = 1/w_av.sum() * (w_av * np.abs(I-I_hat) * np.abs(w_av - w_av.mean())).sum()

In [100]:
print(f'\nweighted average: I_hat: {I_hat}, variance v_hat: {v_hat}, estimated variance v_est: {v_est}')


weighted average: I_hat: 9.90909090909091, variance v_hat: 1.126831683168317, estimated variance v_est: 0.7438016528925584


Alternatively, we can use inverse-variance weighting on the datapoints to obtain the mean with the smallest variance, according to [6]: 

$$
\hat{I} = \frac{\sum_i I_i / \sigma_i^2}{\sum_i 1/\sigma_i^2} 
$$

which has a variance of
$$ 
\sigma^2 = \frac{1}{\sum_i 1/ \sigma_i^2}
$$


In [101]:
v_av = v_goodman
I_hat = (I / v_av).sum() / (1/v_av).sum()
v_hat = 1 / (1/v_av).sum()
print(f'inverse variance weighted average: I_hat: {I_hat:0.02f}, inverse variance weighted variance: {v_hat:0.02f}')

inverse variance weighted average: I_hat: 9.90, inverse variance weighted variance: 0.94



|         | signal $I$ | variance $v$ | new var. est $v_new$ |
| -----   | ---------- | ------------ | --------------- |
| unweighted average | 9.5        |  5.425       |    0.0          |
|   weighted average |  9.91       |   1.127      |    0.744         |
| inv. var. wt. av.  | 9.90        | 0.94 | - |

#### Literature sources: 
  1. https://veritas.ucd.ie/~apl/labs_master/docs/2020/DA/Matrix-Methods-for-Error-Propagation.pdf
  2. https://en.wikipedia.org/wiki/Propagation_of_uncertainty
  3. https://ds.ifi.uni-heidelberg.de/files/Team/eschubert/publications/SSDBM18-covariance-authorcopy.pdf
  4. https://www.tandfonline.com/doi/abs/10.1080/01621459.1960.10483369
  5. https://en.wikipedia.org/wiki/Coefficient_of_variation
  6. https://en.m.wikipedia.org/wiki/Inverse-variance_weighting
  7. https://journals.iucr.org/j/issues/2025/01/00/jo5109/index.html

Apropos, this one is wild, and might be worth a decent look: 
@MISC {454266,
    TITLE = {How can I calculate uncertainty of the mean of a set of samples with different uncertainties?},
    AUTHOR = {whuber (https://stats.stackexchange.com/users/919/whuber)},
    HOWPUBLISHED = {Cross Validated},
    NOTE = {URL:https://stats.stackexchange.com/q/454266 (version: 2022-08-02)},
    EPRINT = {https://stats.stackexchange.com/q/454266},
    URL = {https://stats.stackexchange.com/q/454266}
}

## Comparison with Jerome's equations

Let's check if we are compatible with the method implemented in PyFAI for the weight-averaged intensity (eq. 2 in [7]): 
$$
\hat{I_J} = \frac{\sum_i I_i}{\sum_i \mathrm{norm_i}} 
$$
where $\mathrm{norm_i}$ contains the normalizations (pixel area in our example), and: 
$$ 
\hat{\sigma_J^2} = \frac{\sum_i \sigma^2_i}{\sum_i \mathrm{norm_i}} 
$$


In [108]:
I  = np.array([9, 100], dtype=float) # signal
v  = np.array([9, 100], dtype=float) # variance of signal
w  = np.array([1, 10], dtype=float) # weights
I_hat_J = I.sum() / w.sum()
v_hat_J = v.sum() / w.sum()
print(f"Jerome's method: I_hat_J: {I_hat_J:0.02f}, v_hat_J: {v_hat_J:0.02f}")

Jerome's method: I_hat_J: 9.91, v_hat_J: 9.91


# on BaseData
BaseData has become somewhat unwieldy for calculations / modules, with three sets of units, and a nonintuitive use of normalization. 

I therefore propose the following structure: 

BaseData:
 - Data elements:
   - signal: np.ndarray
   - variances: dict (should we define some default names?)
   - weights: 1 (default) or np.ndarray when set, only used for weighted averaging operations
   - mask: 0 (default) or np.ndarray, dtype uint8 of size signal, only used for skipping datapoints when not 0
   - scalar: float, default 1
   - scalar_variance: float, default 0
 - Metadata elements:
   - units: pint.Unit, for signal*scalar
   - rank_of_data: int, default: min(2, signal.ndim)
   - axes: list[Self|None], points at other BaseData elements in the DataBundle (when available, what to do with things like "temperature" or "stage position"?). 

For single values read in from metadata elements in the files, the following must be set:
BaseData (single element):
 - Data elements:
   - signal: float (will act on scalar for multiplication/division operations, otherwise on signal)
   - variances: {'var': float}
 - Metadata elements:
   - units: pint.Unit
   - rank_of_data: 0 (set automatically when signal is not np.ndarray)

Implementation notes:
* operations should act between BaseData and BaseData objects
* weights will be used for weighted averaging, and can be set to 1/normalization for Jerome's algorithm. weighted averages are normalized to 1/(sum(weights)) to avoid impacting the scaling. 
* an alternative averaging step can be weighted by the inverse variance of the datapoint. See reference [6]
* a separate step will apply scalar to the signal, after which scalar and scalar_variance are normalized to scalar. 
* there is only one units to take care of, that's the units of the signal*scalar
* the propagation of variances in a weighted averaging step is apparently not unambiguously defined. this might need some thought. 
* we will have a library of basic maths operations (multiplication, division, addition and subtraction, as well as some trigionometric methods for calculating q, psi) which will have variance propagation implemented. 
* whether it is more beneficial to propagate variance or uncertainty depends on the mathematical operation. variance is, however, more unambiguously defined. 


In [124]:
Q = np.array([[1, 10, 12], [1,2,3]], dtype=float) # area to normalize by
q = np.array(4, dtype=float) # weights for area

p = np.array([2, 3, 4], dtype=float) # signal
Q*p

array([[ 2., 30., 48.],
       [ 2.,  6., 12.]])

In [125]:
np.broadcast_shapes(Q.shape, p.shape)  # This will raise an error if the shapes are not compatible for broadcasting

(2, 3)